<a href="https://colab.research.google.com/github/RMoraffah/NLP-Papers/blob/master/Copy_of_Final_CSE576_Quail_Adversarial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#_________________________
# Finetune Pretrained Models for QuAIL

QuAIL: http://text-machine.cs.uml.edu/lab2/projects/quail/

Install forked HF transformers repo :)

In [ ]:
!rm -rf transformers/
!git clone https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Cloning into 'transformers'...
remote: Enumerating objects: 49787, done.
remote: Total 49787 (delta 0), reused 0 (delta 0), pack-reused 49787
Receiving objects: 100% (49787/49787), 64.19 MiB | 23.47 MiB/s, done.
Resolving deltas: 100% (34724/34724), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 30.1MB/s 
     |████████████████████████████████| 2.9MB 63.0MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1297093 sha256=8bb25e2875d6166ae1b63a8de58e31e91e5ab77141bb5121bb31daf815bfadca
  Stored in directory: /tmp/pip-ephem-wheel-cache-931d85eq/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any

     |████████████████████████████████| 17.7MB 216kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
/content


# New Section

Mount google drive for direct file access if desired.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

Download QuAIL data:

In [ ]:
!mkdir quail
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/train.jsonl -O quail/train.jsonl
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/dev.jsonl -O quail/dev.jsonl
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/challenge.jsonl -O quail/challenge.jsonl

mkdir: cannot create directory ‘quail’: File exists
--2020-11-30 23:30:24--  https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24941234 (24M) [text/plain]
Saving to: ‘quail/train.jsonl’

quail/train.jsonl   100%[===================>]  23.79M  51.1MB/s    in 0.5s    

2020-11-30 23:30:27 (51.1 MB/s) - ‘quail/train.jsonl’ saved [24941234/24941234]

--2020-11-30 23:30:27--  https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/dev.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.

# New Section

# New Section

# New Section

Initialize wandb to track and monitor the training session:

In [ ]:
!pip install wandb
import wandb
wandb.init()

wandb: Currently logged in as: ngdodd (use `wandb login --relogin` to force relogin)


#_________________________

#Training

**Baseline Bert** (from QuAIL paper):

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path bert-base-uncased \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 400 \
  --output_dir quail_out/bert_base_uncased_400 \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

**RoBERTa-Base**

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-swag_hellaswag_cosmos_qa \
  --logging_steps 50

2020-11-24 06:53:01.784676: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/24/2020 06:53:03 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/24/2020 06:53:03 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov24_06-53-03_3bb24e0696eb', log

**RoBERTa-Large**

In [ ]:
 !python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-large \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_large \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

**Longformer**:

Install longformer:

In [ ]:
pip install git+https://github.com/allenai/longformer.git

Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path allenai/longformer-base-4096 \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 650 \
  --output_dir quail_out/longformer_base_4096 \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

**RoBERTA** (using Quail dataset originally in the Swag dataset format)

Note: the only thing different here is that the source dataset files used are in the Swag dataset format. Prior to the training loop, the script will parse/format, tokenize, and then cache the entire train and dev datasets into a common data structure that is used across all of the data Processors. This was done originally due to there not being a data Processor available for QuAIL in huggingface/transformers out of the box. The prior RoBERTa training block above uses the newly introduced QuAIL data Processor, and results for the two should be equivalent.

Reformat Quail data into the Swag format for use with the multiple choice script's Swag data processor:

In [ ]:
import pandas as pd
from datasets import load_dataset

# Load and process the quail train, val, and test datasets
quail = load_dataset('quail').map(lambda e: e.update({'a':e['answers'][0], 'b':e['answers'][1], 'c':e['answers'][2], 'd':e['answers'][3]}) or e)

# Convert and write quail data to swag formatted csv files
reindex = ['context_id', 'id', 'domain', 'context', 'question', 'question_id', 'a', 'b', 'c', 'd', 'correct_answer_id']
pd.DataFrame(list(quail['train'])).reindex(columns=reindex).to_csv('quail/train.csv')
pd.DataFrame(list(quail['validation'])).reindex(columns=reindex).to_csv('quail/val.csv')
pd.DataFrame(list(quail['challenge'])).reindex(columns=reindex).to_csv('quail/test.csv')

Train:

In [ ]:
 !python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name swag \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base_swag \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

#Results Summary

Model Name | Eval Accuracy
--- | ---
Baseline BERT | 0.56
Roberta | 0.65
Longformer | 0.68

#Inference

Test with pre-trained model (no-finetuning)

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

#Experimental

#### Deberta for MCQ

Delete any current transformers repo and clone+install the deberta mcq branch:

In [ ]:
!rm -rf transformers
!git clone -b deberta-for-multiple-choice https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Cloning into 'transformers'...
remote: Enumerating objects: 49872, done.
remote: Total 49872 (delta 0), reused 0 (delta 0), pack-reused 49872
Receiving objects: 100% (49872/49872), 129.43 MiB | 11.19 MiB/s, done.
Resolving deltas: 100% (34768/34768), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 5.8MB/s 
     |████████████████████████████████| 890kB 67.3MB/s 
     |████████████████████████████████| 1.1MB 50.5MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1297093 sha256=0cd7ac0073960147cb886f9e299d3360aa9e34ed6237c36c70ee6f943437f54d
  Stored in directory: /tmp/pip-ephem-wheel-cache-zqtqlrs_/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any

Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path microsoft/deberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/deberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

#### DeBERTa MCQ with Reasoning Task Labels

Delete any current transformers repo and clone+install the mcq-with-reasoning-type branch:




In [ ]:
!rm -rf transformers
!git clone -b mcq-with-reasoning-type https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path microsoft/deberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/deberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --with_reasoning_types

#### RoBERTa-Base MCQ with Reasoning Task Labels



Delete any current transformers repo and clone+install the roberta-mcq-with-reasoning-types branch:

In [ ]:
!rm -rf transformers
!git clone -b roberta-mcq-with-reasoning-types https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-swag+hellaswag+cosmos_qa \
  --logging_steps 50

#### Cross domain MCQ with 

Preprocess the data and add the domain_label

In [ ]:
!mkdir quail
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/train.jsonl -O quail/train.jsonl
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/dev.jsonl -O quail/dev.jsonl
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/challenge.jsonl -O quail/challenge.jsonl

--2020-11-30 15:54:33--  https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24941234 (24M) [text/plain]
Saving to: ‘quail/train.jsonl’

quail/train.jsonl   100%[===================>]  23.79M  48.9MB/s    in 0.5s    

2020-11-30 15:54:39 (48.9 MB/s) - ‘quail/train.jsonl’ saved [24941234/24941234]

--2020-11-30 15:54:39--  https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/dev.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [ ]:
import pandas as pd
import json

In [ ]:
def read_data(path):
  with open(path) as f:
        lines = f.read().splitlines()
  df_inter = pd.DataFrame(lines)
  df_inter.columns = ['json_element']
  df_inter['json_element'].apply(json.loads)
  df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))
  return df_final

In [ ]:
def preprocess_data(path_source, path_target):
    df_source = read_data(path_source)  
    df_source ['domain_type'] = "1"
    df_target = read_data(path_target)  
    df_target ['domain_type'] = "0"
    df_final = pd.concat([df_source, df_target])
    return df_final

In [ ]:
!mkdir quail_adv

In [ ]:
df_train = preprocess_data("quail_adv/synthetic_with_nei_cleaned_without_sequential_train.jsonl", "quail/train.jsonl")
#df_train.to_csv("quail_adv/train.csv")
df_train.to_json("quail_adv/train.jsonl", orient='records', lines=True)
df_dev = preprocess_data("quail_adv/synthetic_with_nei_cleaned_without_sequential_dev.jsonl", "quail/dev.jsonl")
#df_dev.to_csv("quail_adv/dev.csv")
df_dev.to_json("quail_adv/dev.jsonl", orient='records', lines=True)

In [ ]:
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/challenge.jsonl -O quail_adv/challenge.jsonl

--2020-11-30 15:55:56--  https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/challenge.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1282446 (1.2M) [text/plain]
Saving to: ‘quail_adv/challenge.jsonl’

quail_adv/challenge 100%[===================>]   1.22M  --.-KB/s    in 0.08s   

2020-11-30 15:55:56 (15.8 MB/s) - ‘quail_adv/challenge.jsonl’ saved [1282446/1282446]



Delete any current transformers repo and clone+install the roberta-adversarial branch:

In [ ]:
!rm -rf transformers
!git clone -b roberta-adversarial https://github.com/RMoraffah/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Cloning into 'transformers'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 49996 (delta 35), reused 0 (delta 0), pack-reused 49931
Receiving objects: 100% (49996/49996), 64.26 MiB | 11.34 MiB/s, done.
Resolving deltas: 100% (34847/34847), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 4.2MB/s 
     |████████████████████████████████| 2.9MB 19.5MB/s 
     |████████████████████████████████| 890kB 34.7MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1299140 sha256=a08187be267c388a58ed5e0893743c9d6274d2908147074ab90fdeae46771a07
  Stored in directory: /tmp/pip-ephem-wheel-cache-tbe7fkwo/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully

     |████████████████████████████████| 17.7MB 212kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
/content


Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail_adv \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-adversarial \
  --logging_steps 50 \
  --with_adv_training \
  --alpha 1

2020-11-30 00:45:51.076211: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 00:45:53 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/30/2020 00:45:53 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_00-45-53_9254f8c29b62', log

# Test on small scale

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail_adv \
  --learning_rate 1e-5 \
  --num_train_epochs 1 \
  --max_seq_length 2 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-adversarial \
  --logging_steps 50 \
  --with_adv_training \
  --alpha 0.1


2020-11-30 06:24:24.740575: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 06:24:26 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/30/2020 06:24:26 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_06-24-26_1165d2d919ea', log

# Different paramter configurations to make the performance better :)

In [ ]:
from google.colab import files
#For the folder you have to zip it first and can only download later on
!zip -r results.zip quail_out
#Download files
files.download('results.zip')

  adding: quail_out/ (stored 0%)
  adding: quail_out/roberta_base/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Config 2: alpha = 0.00001

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail_adv \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-adversarial \
  --logging_steps 50 \
  --with_adv_training \
  --alpha 0.00001

# Config 3: alpha + epoch

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail_adv \
  --learning_rate 1e-5 \
  --num_train_epochs 3 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-adversarial \
  --logging_steps 50 \
  --with_adv_training \
  --alpha 0.00001

2020-11-30 06:41:04.439365: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 06:41:06 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/30/2020 06:41:06 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_06-41-06_1165d2d919ea', log

In [ ]:
from google.colab import files
#For the folder you have to zip it first and can only download later on
!zip -r results.zip quail_out
#Download files
files.download('results.zip')

updating: quail_out/ (stored 0%)
updating: quail_out/roberta_base/ (stored 0%)
  adding: quail_out/roberta_base/training_args.bin (deflated 45%)
  adding: quail_out/roberta_base/checkpoint-1500/ (stored 0%)
  adding: quail_out/roberta_base/checkpoint-1500/training_args.bin (deflated 45%)
  adding: quail_out/roberta_base/checkpoint-1500/scheduler.pt (deflated 49%)
  adding: quail_out/roberta_base/checkpoint-1500/trainer_state.json (deflated 73%)
  adding: quail_out/roberta_base/checkpoint-1500/pytorch_model.bin (deflated 10%)
  adding: quail_out/roberta_base/checkpoint-1500/config.json (deflated 53%)
  adding: quail_out/roberta_base/checkpoint-1500/optimizer.pt (deflated 18%)
  adding: quail_out/roberta_base/checkpoint-2500/ (stored 0%)
  adding: quail_out/roberta_base/checkpoint-2500/training_args.bin (deflated 45%)
  adding: quail_out/roberta_base/checkpoint-2500/scheduler.pt (deflated 49%)
  adding: quail_out/roberta_base/checkpoint-2500/trainer_state.json (deflated 75%)
  adding: qu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Config 4: alpha = 0.001

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail_adv \
  --learning_rate 1e-5 \
  --num_train_epochs 3 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-adversarial \
  --logging_steps 50 \
  --with_adv_training \
  --alpha 0.001

2020-11-30 15:57:13.105869: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 15:57:15 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/30/2020 15:57:15 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_15-57-15_2bd1100af53f', log

In [ ]:
!mkdir quail_test
df_dev = read_data("quail/dev.jsonl")
df_dev ['domain_type'] = "0"
df_dev.to_json("quail_test/dev.jsonl", orient='records', lines=True)


mkdir: cannot create directory ‘quail_test’: File exists


# Run the previous config on quail-dev

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path /content/quail_out/roberta_base  \
  --do_eval \
  --data_dir ./quail_test \
  --learning_rate 1e-5 \
  --num_train_epochs 3 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-adversarial-test \
  --logging_steps 50 \
  --with_adv_training \
  --alpha 0.001

2020-11-30 23:31:11.976110: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 23:31:13 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/30/2020 23:31:13 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=False, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_23-31-13_2bd1100af53f', lo

In [ ]:
from google.colab import files
!zip -r results.zip quail_out
#Download files
files.download('results.zip')